In [1]:
import ROOT
from ROOT import *
import copy
import plot_settings as plotsetting

Welcome to JupyROOT 6.24/02


 Get the data and background histogram files
 
 Make a dictionary contains of data and MC histo files for later

In [2]:
inputfilepath='../inputfiles/'

WorkdayTag= '_Sep6_v0'

AnalysisName= 'VLL2018_'

dataAndMCbundle=['Data','wjets','zjets','ttjets','tW','STtchannelTop','WW2Q','QCDMultijet']
FileName=['data', 'wjets', 'zjets', 'ttjets', 'tW', 'ST', 'WW2Q','QCD']

In [3]:
FilesDict={}
for ifile,file in enumerate(dataAndMCbundle):
    file_ifile=TFile.Open(inputfilepath+AnalysisName+dataAndMCbundle[ifile]+WorkdayTag+'.root','READ')
    FilesDict[FileName[ifile]]=file_ifile
        
FilesDict

{'data': <cppyy.gbl.TFile object at 0x71091d0>,
 'wjets': <cppyy.gbl.TFile object at 0x4365020>,
 'zjets': <cppyy.gbl.TFile object at 0x4365760>,
 'ttjets': <cppyy.gbl.TFile object at 0x44be460>,
 'tW': <cppyy.gbl.TFile object at 0x21782f0>,
 'ST': <cppyy.gbl.TFile object at 0x21526c0>,
 'WW2Q': <cppyy.gbl.TFile object at 0x2114f70>,
 'QCD': <cppyy.gbl.TFile object at 0x8880540>}

In [4]:
CrossSectionDB={
    'data':1,
    'wjets':50380,
    'zjets':5765,
    'ttjets':365.34,
    'tW':34.91,
    'ST':136.02,
    'WW2Q':45.68,
    'QCD':239400    
}

OverlayColorScheme={
    'data':kBlack,
    'wjets':kGray+1,
    'zjets':kRed-9,
    'ttjets':kCyan-10,
    'tW':kBlue-9,
    'ST':kBlue-7,
    'WW2Q':kGreen-4,
    'QCD':kYellow    
}

StackingColorScheme={
    'data':kBlack,
    'wjets':kGray+1,
    'zjets':kRed-9,
    'ttjets':kCyan-10,
    'tW':kBlue-9,
    'ST':kBlue-5,
    'WW2Q':kGreen-10,
    'QCD':kYellow-10    
}                 

Let's define a function which will make a histogram dictionary from the TFiles. The Dictionary would look like,
>**Key:** SampleName
<br>
>**Value:** List(SampleName,RawEvent,LumiCorrectedEvent,HistoColor,HistogramPointerObject)

The histos are produced by NanoAOD code as the following,
> - **WJetCR Normal:** ""
<br>
> - **MVA Region:** "_highMVA"
<br><br>
> *Used for ABCD method*
<br>
> - **Region A:** "_regA"
<br>
> - **Region B:** "_regB"
<br>
> - **Region C:** "_regC"
<br>
> - **Region D:** "_regD"

So, typically a histogram name is,

**Plotname=** VariableName + '_l2j_' + Region,    Such as, *HT_l2j_regA*

In [5]:
def GetHistoDict(region,var):

    HistoDict={}
    #histogram=[]
    dataLumi=59.7*1000 #in pb

    #Create Plotname
    plotname=var+'_l2j'+region
    
    for index,(filekey,file) in enumerate(FilesDict.items()):
        #print(filekey,file)
        histo= file.Get(plotname)
        histo.SetDirectory(0)
        histoevent=float(Count(histo))

        #LumiCorr-------------------------------
        totEvent=file.nEvents.GetBinContent(2)
        Xsec=CrossSectionDB[filekey]
        sampleLumi=totEvent/Xsec
        if(filekey!='data'): lumi_corr=dataLumi/sampleLumi
        else: lumi_corr=1
        #print(filekey,totEvent,Xsec,sampleLumi,lumi_corr)    
        histo.Scale(lumi_corr)
        histoeventAfterCorr=float(Count(histo))
        #---------------------------------------
        #histogram.append([filekey,histoevent,histoeventAfterCorr,OverlayColorScheme[filekey],histo])
        HistoDict[filekey]=[filekey,histoevent,histoeventAfterCorr,OverlayColorScheme[filekey],histo]

    return HistoDict
#---------------------------------------------------------------------------------------
#Count Events in histogram
def Count(histo):                                                               
    n_count="{:.3f}".format(histo.Integral())                                   
    return str(n_count)

def Summary(HistoDict):
    ###Sorting based on lumi corrected weight
    #HistoDict=dict(sorted(HistoDict.items(),key=lambda x:x[1][2],reverse=False))
    print("         Summary Report                ")
    print("_________________________________________")
    print("   ")
    print( "  Name"+"     "+"    No of Events ")
    print("--------"+"    "+"-------------------")
    exp=0
    for key in HistoDict.keys():
        if(key!='data'): exp=exp+HistoDict[key][2]
        print(" "+ key+"          " +str(HistoDict[key][2]))
    print("_________________________________________")
    print("    ")
    print("obs="+str(HistoDict['data'][2])+", "+"exp="+str(exp))
    print("obs/exp=",str((HistoDict['data'][2]/exp))[:5])
    print("_________________________________________")
    print("                                         ")
    obs=HistoDict['data'][2]
    bkgBundle=list(HistoDict.keys())[:-1]
    print("Background Bundle=",bkgBundle) 

def PlotMaker(Dictionary,Xaxislabel,rebin,SaveOption): ## Give the dictionary of histograms
    Dict=copy.deepcopy(Dictionary)
    Dict=dict(sorted(Dict.items(),key=lambda x:x[1][2],reverse=False))
    #Background Bundle
    bkgBundle=list(Dict.keys())[:-1]
    print(bkgBundle)
    
    rebinning=rebin
    #DataHisto
    dataHisto=Dict['data'][-1]
    dataHisto.Rebin(rebinning)
    plotsetting.SetOverflowBin(dataHisto)
        
    ##Legend Settings
    legend = ROOT.TLegend(0.99, 0.45, 0.80,0.86) 
    plotsetting.SetLegendStyle(legend)
    legend.AddEntry(dataHisto,"Data ["+plotsetting.Count(dataHisto)+"]","ep")
    for imc,mc in reversed(list(enumerate(bkgBundle))):
        histoname=Dict[mc][0]
        color=Dict[mc][3]
        histo=Dict[mc][-1]    
        legend.AddEntry(histo,histoname+"["+Count(histo)+"]","lf")
        
    ##
    ##Stacking(ALL MC Backgrounds)
    stack=THStack()
    ratioHisto=dataHisto.Clone()
    h_Allbkg=Dict['ST'][-1].Clone()
    h_Allbkg.Rebin(rebinning)
    for imc,mc in enumerate(bkgBundle):
        histoname=Dict[mc][0]
        color=Dict[mc][3]
        histo=Dict[mc][-1]    
        ##SetStyle
        histo.SetLineWidth(0)                                                                                                       
        histo.SetLineColor(1)                                                                                                       
        histo.SetFillColor(color)                                                                                             
        histo.Rebin(rebinning)
        plotsetting.SetOverflowBin(histo)
        stack.Add(histo)                                                                                                            
        #Denominotor for ratioPlot
        if(imc>0): h_Allbkg.Add(histo)
    ratioHisto.Divide(h_Allbkg)
    
    
    obs=float(Count(dataHisto))
    exp=float(Count(h_Allbkg))
    ##Legend Settings
    legend = ROOT.TLegend(0.99, 0.45, 0.80,0.86) 
    plotsetting.SetLegendStyle(legend)
    legend.AddEntry(dataHisto,"Data ["+plotsetting.Count(dataHisto)+"]","ep")
    for imc,mc in reversed(list(enumerate(bkgBundle))):
        histoname=Dict[mc][0]
        color=Dict[mc][3]
        histo=Dict[mc][-1]    
        legend.AddEntry(histo,histoname+"["+Count(histo)+"]","lf")
    ##Obs/Exp Legend
    ratioleg = ROOT.TLegend(0.99, 0.90, 0.81,0.87)
    plotsetting.SetLegendStyle(ratioleg)
    ratioleg.SetTextSize(0.03)
    ratioleg.SetHeader("obs/exp="+(str(obs/exp)[:5])) 
    
    ##Plotting
    cName=plotname
    canvas=TCanvas(cName,cName,700,600)                                                                                             
    pad=plotsetting.CreatePad("pad",0,1.0)                                                                                          
    #pad.SetLogy(0)
    pad.Draw()                                                                                                                      

    #RatioPad                                                                                                                       
    if(plotsetting.PlotRatioPad==True):                                                                                                         
        ratioPad = plotsetting.CreateRatioPad('ratioPad',0,0.25)                                                                    
        ratioPad.SetGrid(1)                                                                                                         
        ratioPad.Draw()                                                                                                             

    pad.cd()                                                                                                                        
    ##MainPadStyle 
    pad.SetFillStyle(4000)
    plotsetting.SetHistoStyle(dataHisto,xlabel="")
    dataHisto.GetYaxis().SetRangeUser(0.1,1e7)
    ##Draw                                                                                                                          
    dataHisto.Draw("ep")                                                                                                            
    stack.Draw("HIST SAME")                                                                                                         
    dataHisto.Draw("ep same")                                                                                                       
    legend.Draw("SAME")
    ratioleg.Draw("SAME")
    plotsetting.CMSLabel(pad)
    pad.SetTickx(1)
    pad.Update()
    ##RatioPadDraw                                                                                                                  
    if(plotsetting.PlotRatioPad==True):                                                                                                         
        ratioPad.cd()                                                                                                               
        plotsetting.SetRatioHistoStyle(ratioHisto,xlabel=Xaxislabel)                                                              
        ratioHisto.Draw('ep')

    #Extra Text
    pad.cd()
    plotsetting.DrawText(0.60,0.84,'2018 WJets CR')    
    
    canvas.Draw()
    if(SaveOption==True):
        canvas.Print(outputfilepath+outputPDFName)
        canvas.SaveAs(outputfilepath+plotname+'.pdf')
        
    return canvas,stack,legend,ratioHisto,ratioleg    

Before Plot Let Set some global plotting parameter

In [6]:
plotsetting.PlotRatioPad=True
plotsetting.year=2018                                                                                                           
plotsetting.publicationStyle=False
plotsetting.WorkInProgress=True

In [7]:
import time

In [8]:
timex=time.ctime()
print('_'.join(timex.split()))
timestamp= '_'.join(timex.split())

Thu_Feb_24_21:18:47_2022


In [9]:
import os
import sys

outputdir=AnalysisName+WorkdayTag+'_'+timestamp
print(outputdir)
os.system('mkdir ../output/'+outputdir)

outputfilepath = '/home/arnab/Arnab/Work/PhD2022/VLLAnalysis/BackgroundStudy/wjet/output/'+outputdir+'/'

VLL2018__Sep6_v0_Thu_Feb_24_21:18:47_2022


In [10]:
outputfilepath

'/home/arnab/Arnab/Work/PhD2022/VLLAnalysis/BackgroundStudy/wjet/output/VLL2018__Sep6_v0_Thu_Feb_24_21:18:47_2022/'

In [11]:
VariableName=["ptlep0","ptjet0","ptjet1","mtlep0","mtjet0","mtjet1","dijetmass","drjet01",
              "dijetPT","dijetMT","MET","HT","ST","njet","nbjet","dphimetjet0","dphimetjet1",
              "dphimetlep0","dphijet0lep0","dphijet1lep0","dphidijetlep0","dphimetdijet"]

RegionName=["","_regA","_regB","_regC","_regD"]
  
LabelName=["Lepton PT","Leading jet Pt","SubLeading jet pt","Lepton MT","Leading Jet MT",
           "Subleading Jet MT","Dijet Inv Mass","deltaR(jet0,jet1)","Dijet System Inv Mass",
           "Dijet System MT","MET","HT","ST","No of jets","No of b-jets","dphi(met,leading jet)",
           "dphi(met,subleading jet)","dphi(met,lepton)","dphi(jet0,lepton)","dphi(jet1,lepton)",
           "dphi(dijet system,lepton)","dphi(dijet system,met)"]

In [12]:
binning=[20,20,20,25,25,25,25,5,25,25,25,25,25,1,1,5,5,5,5,5,5,5]

In [13]:
outputPDFName='WjetCRPlot.pdf'
c=TCanvas()
c.Print(outputfilepath+outputPDFName+"[")
for ireg,reg in enumerate(RegionName):
    for ivar,var in enumerate(VariableName):
        plotname=var+'_l2j'+reg
        
        TempoDict=GetHistoDict(reg,var)
        
        XaxisName=LabelName[ivar]+reg
        rebin=binning[ivar]
        PlotMaker(TempoDict,XaxisName,rebin,True)
        TempoDict.clear()
c.Print(outputfilepath+outputPDFName+"]")        

['ST', 'tW', 'WW2Q', 'ttjets', 'zjets', 'QCD', 'wjets']
['ST', 'tW', 'WW2Q', 'ttjets', 'zjets', 'QCD', 'wjets']
['ST', 'tW', 'WW2Q', 'ttjets', 'zjets', 'QCD', 'wjets']
['ST', 'tW', 'WW2Q', 'ttjets', 'zjets', 'QCD', 'wjets']
['ST', 'tW', 'WW2Q', 'ttjets', 'zjets', 'QCD', 'wjets']
['ST', 'tW', 'WW2Q', 'ttjets', 'zjets', 'QCD', 'wjets']
['ST', 'tW', 'WW2Q', 'ttjets', 'zjets', 'QCD', 'wjets']
['ST', 'tW', 'WW2Q', 'ttjets', 'zjets', 'QCD', 'wjets']
['ST', 'tW', 'WW2Q', 'ttjets', 'zjets', 'QCD', 'wjets']
['ST', 'tW', 'WW2Q', 'ttjets', 'zjets', 'QCD', 'wjets']
['ST', 'tW', 'WW2Q', 'ttjets', 'zjets', 'QCD', 'wjets']
['ST', 'tW', 'WW2Q', 'ttjets', 'zjets', 'QCD', 'wjets']
['ST', 'tW', 'WW2Q', 'ttjets', 'zjets', 'QCD', 'wjets']
['ST', 'tW', 'WW2Q', 'ttjets', 'zjets', 'QCD', 'wjets']
['ST', 'tW', 'WW2Q', 'ttjets', 'zjets', 'QCD', 'wjets']
['ST', 'tW', 'WW2Q', 'ttjets', 'zjets', 'QCD', 'wjets']
['ST', 'tW', 'WW2Q', 'ttjets', 'zjets', 'QCD', 'wjets']
['ST', 'tW', 'WW2Q', 'ttjets', 'zjets', 'QCD', '

Info in <TCanvas::Print>: pdf file /home/arnab/Arnab/Work/PhD2022/VLLAnalysis/BackgroundStudy/wjet/output/VLL2018__Sep6_v0_Thu_Feb_24_21:18:47_2022/WjetCRPlot.pdf has been created
Info in <TCanvas::Print>: Current canvas added to pdf file /home/arnab/Arnab/Work/PhD2022/VLLAnalysis/BackgroundStudy/wjet/output/VLL2018__Sep6_v0_Thu_Feb_24_21:18:47_2022/WjetCRPlot.pdf
Info in <TCanvas::Print>: pdf file /home/arnab/Arnab/Work/PhD2022/VLLAnalysis/BackgroundStudy/wjet/output/VLL2018__Sep6_v0_Thu_Feb_24_21:18:47_2022/ptlep0_l2j.pdf has been created
Info in <TCanvas::Print>: Current canvas added to pdf file /home/arnab/Arnab/Work/PhD2022/VLLAnalysis/BackgroundStudy/wjet/output/VLL2018__Sep6_v0_Thu_Feb_24_21:18:47_2022/WjetCRPlot.pdf
Info in <TCanvas::Print>: pdf file /home/arnab/Arnab/Work/PhD2022/VLLAnalysis/BackgroundStudy/wjet/output/VLL2018__Sep6_v0_Thu_Feb_24_21:18:47_2022/ptjet0_l2j.pdf has been created
Info in <TCanvas::Print>: Current canvas added to pdf file /home/arnab/Arnab/Work/PhD2

In [14]:
#Sample plot


In [15]:
var='ST'
TempDict=GetHistoDict('',var)

In [16]:
TempDict

{'data': ['data',
  2196642.0,
  2196642.0,
  1,
 'wjets': ['wjets',
  24353.0,
  1276011.838,
  921,
 'zjets': ['zjets',
  32634.0,
  112634.347,
  623,
 'ttjets': ['ttjets',
  416180.0,
  98647.673,
  422,
 'tW': ['tW', 45608.0, 9903.403, 591, <cppyy.gbl.TH1F object at 0x9c3f2e0>],
 'ST': ['ST', 7023.0, 4085.707, 593, <cppyy.gbl.TH1F object at 0x9c3e6e0>],
 'WW2Q': ['WW2Q',
  41516.0,
  24175.706,
  412,
 'QCD': ['QCD', 1018.0, 656405.559, 400, <cppyy.gbl.TH1F object at 0x9c4ed50>]}

In [17]:
%jsroot
#TempDict
PlotMaker(TempDict,var,25,False)
#TempDict.clear()

['ST', 'tW', 'WW2Q', 'ttjets', 'zjets', 'QCD', 'wjets']


(<cppyy.gbl.TCanvas object at 0x6bf6430>,
 <cppyy.gbl.TLegend object at 0x9eae0b0>)

In [18]:
TempDict.clear()